In [ ]:
!pip install datasets

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# with open('input.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

In [ ]:
torch.cuda.empty_cache()
torch.cuda.amp.autocast(enabled=True)

In [3]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

In [6]:
from datasets import load_dataset

dataset = load_dataset("roneneldan/TinyStories")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [13]:
text = ""
numRows = 0
for row in dataset['train']:
    datatext = row['text']
    datatext = datatext.encode('ascii',errors='ignore').decode()
    text = text + str.replace(datatext, '\n', ' ')
    numRows = numRows + 1
    if numRows % 10000 == 0:
        print(numRows)
    if numRows > 50000:
        break

print(len(text))
print(text[-1000:-1])

10000
20000
30000
40000
50000
44515910
ls. She practiced wheelies, jumps and tricks. Sometimes, she would make it all the way to the park and have lots of fun. But, one day the scooter broke. She worked hard to fix it. Finally, with a little help from her dad, it was working again. She was so happy and went back to playing and doing scooter tricks again. She was so proud of her hard work!Once upon a time there was a big room. In this room, there were two boys. One boy was proud and the other wasn't.  All of a sudden, the proud boy asked the other boy to freeze! He was so proud that he wanted the other boy to stand perfectly still. But the other boy wasn't so sure.  The proud boy counted to three and then shouted 'freeze!' On that same word, the other boy became as still as a statue, standing tall and proud.  The proud boy watched and smiled. He was so proud of his friend for freezing so well, that he wanted to give him a big hug. So he did, and both boys smiled and laughed.  They were 

In [14]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [15]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [16]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [17]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [18]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

#Tensorboard
writer = SummaryWriter(config["experiment_name"])

# Only 1 batch per epoch
STEPS_PER_EPOCH = 1

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    epochs=max_iters,
    steps_per_epoch=STEPS_PER_EPOCH,
    pct_start=0.3,
    div_factor=100,
    final_div_factor=100,
    three_phase=False,
    anneal_strategy='linear',
    verbose=False
)

scaler = torch.cuda.amp.grad_scaler.GradScaler()
lrVals = [0.0]

for iter in range(max_iters):

    torch.cuda.empty_cache()
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, lr {lrVals[-1]:4.8f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        logits, loss = m(xb, yb)

    #Backpropogate loss
    scaler.scale(loss).backward()
    
    scale = scaler.get_scale()
    scaler.step(optimizer)
    scaler.update()
    skip_lr_sched = (scale > scaler.get_scale())
    if not skip_lr_sched:
        scheduler.step()
    lrVals.append(scheduler.get_last_lr()[0])
    
    optimizer.zero_grad(set_to_none=True)

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.211922 M parameters
step 0: train loss 4.6078, val loss 4.6064
step 100: train loss 2.4858, val loss 2.4901
step 200: train loss 2.3518, val loss 2.3439
step 300: train loss 2.2287, val loss 2.2211
step 400: train loss 2.1138, val loss 2.1234
step 500: train loss 2.0444, val loss 2.0483
step 600: train loss 1.9626, val loss 1.9692
step 700: train loss 1.8922, val loss 1.9009
step 800: train loss 1.8447, val loss 1.8475
step 900: train loss 1.7947, val loss 1.8074
step 1000: train loss 1.7376, val loss 1.7363
step 1100: train loss 1.7110, val loss 1.7162
step 1200: train loss 1.6746, val loss 1.6773
step 1300: train loss 1.6417, val loss 1.6564
step 1400: train loss 1.6283, val loss 1.6359
step 1500: train loss 1.5994, val loss 1.6126
step 1600: train loss 1.5733, val loss 1.5862
step 1700: train loss 1.5629, val loss 1.5719
step 1800: train loss 1.5490, val loss 1.5586
step 1900: train loss 1.5296, val loss 1.5426
step 2000: train loss 1.5089, val loss 1.5316
step 2100: train loss 1.

In [19]:
torch.save(m.state_dict(), 'trainedstorycharmodel.pt')